In [ ]:
from pathlib import Path

import numpy as np
import pyshtools as pysh

import matplotlib.pyplot as plt

from redplanet.DatasetManager.main import _get_fpath_dataset
from redplanet.DatasetManager.hash import _calculate_hash_from_file
from redplanet.helper_functions.misc import timer

In [ ]:
r0_km = 3396000.0 / 1.e3

# '''lmax -> res'''
# lmax = 1439
# res = (2 * np.pi * r0_km) / (2 * lmax + 1)
# print(f'{lmax = }')
# print(f'-> resolution = {res:.2f} km')

'''res -> lmax'''
res_km = 10
lmax = int((((2 * np.pi * r0_km) / (res_km)) - 1) / 2)
print(f'resolution = {res_km} km')
print(f'-> {lmax = }')

- resolution = 5 km
    - -> lmax = 2133
- resolution = 10 km
    - -> lmax = 1066

In [ ]:
with timer('Load topography: '):

    shape_coeffs = pysh.datasets.Mars.MOLA_shape(lmax=lmax)
    shape_grid = shape_coeffs.expand(grid='DH2')

    grav_coeffs = pysh.datasets.Mars.GMM3()  ## we only care about the r0 value (3396000.0) -- this will download the GMM3 dataset to pyshtools cache, but that's okay since it's only done once locally to generate the datasets => users won't have to download it.
    grav_geoid = grav_coeffs.geoid(
        potref = pysh.constants.Mars.u0.value,
        omega  = pysh.constants.Mars.angular_velocity.value,
        lmax   = lmax,
    )

    topo_grid = shape_grid - (grav_coeffs.r0 + grav_geoid.geoid)


topo_xr = topo_grid.to_xarray()
topo_xr = topo_xr.isel(lat = slice(None, None, -1))  ## reverse lat since pyshtools gives it in decreasing order by default
topo_xr = topo_xr.round().astype(np.int16)  ## float64 gives values like `4394.94769267505` meters, which is precise up to 0.1 angstrom (one-tenth the diameter of a typical hydrogen atom) -- int16 (i.e. nearest meter) is sufficient lol


size_mb = (16 * topo_xr.size) / 8.e6
print(f'size = {size_mb :.0f} MB')

print('---')
topo_xr

In [ ]:
'''preview resolution'''

topo_grid.plot(
    colorbar    = 'bottom',
    cb_label    = 'Topography [m]',
    cmap        = 'RdBu_r',
    cmap_limits = [-8.5e3, 8.e3],
    show        = False,
)
plt.show()


from redplanet import Craters
crater = Craters.get(name='Henry', as_dict=True)[0]
r = 3
topo_xr.sel(
    lon = slice(crater['plon'] - r, crater['plon'] + r),
    lat = slice(crater['lat'] - r, crater['lat'] + r),
).plot.imshow()
plt.show()

In [ ]:
'''metadata for loading'''

print(f'{topo_xr.shape = }')
print(f'{topo_xr.dtype = }')
print()

## pyshtools always gives coords from 0->360 and -90->90 inclusive, so all you need to parameterize the coordinates is the number of points in each dimension

_lons = topo_xr.lon.values
lons = np.linspace(_lons[0], _lons[-1], len(_lons))
assert np.allclose(lons, _lons)
print(f'lons = np.linspace(0, 360, {len(_lons)})')

_lats = topo_xr.lat.values
lats = np.linspace(_lats[0], _lats[-1], len(_lats))
assert np.allclose(lats, _lats)
print(f'lats = np.linspace(-90, 90, {len(_lats)})')

- 5km:
    - topo_xr.shape = (4269, 8537)
    - topo_xr.dtype = dtype('int16')
    - lons = np.linspace(0, 360, 8537)
    - lats = np.linspace(-90, 90, 4269)
- 10km:
    - topo_xr.shape = (2135, 4269)
    - topo_xr.dtype = dtype('int16')
    - lons = np.linspace(0, 360, 4269)
    - lats = np.linspace(-90, 90, 2135)

In [ ]:
'''save to memmap file'''

dirpath_out = Path.cwd() / 'datasets/Crust/topo/sh/output/'
dirpath_out.mkdir(parents=True, exist_ok=True)
fpath_memmap = dirpath_out / f'Mars_MOLA_topography_{res_km}km.memmap'

memmap_array = np.memmap(
    fpath_memmap,
    mode  = 'w+',
    dtype = topo_xr.dtype,
    shape = topo_xr.shape,
)

memmap_array[:] = topo_xr.values[:]
memmap_array.flush()  ## "Write any changes in the array to the file on disk." (not sure if necessary, just in case)

In [ ]:
print(f'{fpath_memmap.name}')
print(f'{fpath_memmap.stat().st_size / 1.e6 :.0f} MB')

# for alg in get_available_algorithms():
#     print(f'- {alg}: {_calculate_hash_from_file(fpath_memmap, alg)}')

alg = 'xxh3_64'
calculated_hash = _calculate_hash_from_file(fpath_memmap, alg)

print(f'- {alg}: {calculated_hash}')

- Mars_MOLA_topography_5km.memmap
    - 73 MB
    - xxh3_64: c342253082d163dd
- Mars_MOLA_topography_10km.memmap
    - 18 MB
    - xxh3_64: 3a8d69acce1a0799